In [4]:
import lightning as L
import numpy as np
import torch

from src.config import compose_config
from sklearn.metrics import f1_score                  
from tqdm import tqdm

from datamodule import Datamodule
from lit import Lit

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vsevo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vsevo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def predict(logits: torch.Tensor) -> np.ndarray:
    """Helper function for predictions calculating.

    Args:
        logits (torch.Tensor): model's raw output

    Returns:
        np.ndarray: array with predicted class id.
    """
    s = torch.nn.Softmax()
    probs = s(torch.tensor(logits))
    return np.argmax(probs)

In [6]:
def get_score(datamodule, lit_module):
    
    datamodule.setup(stage="test")

    preds_logits = torch.tensor([])
    targets = torch.tensor([])

    with torch.no_grad():
        for val_input, val_label in tqdm(datamodule.test_dataloader()):
            mask = val_input['attention_mask']
            input_id = val_input['input_ids'].squeeze(1)
            output = lit_module.model(input_id, mask)[0]
            preds_logits = torch.cat((preds_logits, output))
            targets = torch.cat((targets, val_label.long().cpu()))
    preds = np.apply_along_axis(predict, 1, preds_logits)
    return  f1_score(targets, preds, average='macro')  

In [8]:


cfg = compose_config()

c:\Users\vsevo\MKN\skillbox-practice-vsevolod-lavrov\.pixi\env\Lib\site-packages\hydra\_internal\defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [10]:
datamodule = Datamodule(cfg)
    
# Create lit module
lit_module = Lit(
    cfg
)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Create trainer
trainer = L.Trainer(
    accelerator=cfg['training']['accelerator'],
    max_epochs=cfg['training']['num_train_epochs'], 
)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
# Fit
print('START FITTING')
trainer.fit(
    model=lit_module,
    datamodule=datamodule,
)



START FITTING


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 29.2 M
1 | loss  | CrossEntropyLoss              | 0     
--------------------------------------------------------
29.2 M    Trainable params
0         Non-trainable params
29.2 M    Total params
116.780   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\vsevo\MKN\skillbox-practice-vsevolod-lavrov\.pixi\env\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
c:\Users\vsevo\MKN\skillbox-practice-vsevolod-lavrov\.pixi\env\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step
train_step

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [13]:
# Inference
score = get_score(datamodule, lit_module)
print(f"SCORE: {score}")

 96%|█████████▌| 519/540 [22:18<00:52,  2.49s/it]